 ## Learning Objectives
 
Explore how changing data generation parameters can reveal variance in an industry grade computer vision object detecion model.

1. Sign up with the AI-model-adaptation platform, NVIDIA Train, Adapt, Optimize (TAO) and Rendered.ai's data generation engine, Ana.
2. Use a generated dataset to train a ResNet-18 Faster R-CNN model <br>
3. Evaluate several datasets with the model to quantify the sensitivity of object detection accuaracy to object obstruction.

Supporting Resources:<br>
* https://rendered.ai/<br>
* https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/fasterrcnn.htm<br>
* https://docs.nvidia.com/metropolis/TLT/tlt-user-guide/text/object_detection/fasterrcnn.html


 ### Table of Contents
 1. [Virtual Environment](#head-1)<br>
 2. [Configure TAO](#head-2)<br>
    2.2 [Evironment Variables](#head-2-2)
 3. [Download RenderedAI Datasets](#head-3)<br>
     3.1 [Authenticate with Rendered AI SDK](#head-3-1)<br>
     3.2 [Download and Verify Dataset and kitti format annotations](#head-3-2)<br>
 4. [Prepare dataset and pretrained model](#head-4)<br>
     4.1 [Prepare tfrecords from kitti format dataset](#head-4-1)<br>
     4.2 [Download pretrained model](#head-4-2)
 5. [Run TAO training](#head-5) <br>
     5.1 [Provide training specification](#head-5-1)
 6. [Evaluate trained models](#head-6)
 7. [Visualize inferences](#head-7)

 ## 1. Environment for Local Experiment <a class="anchor" id="head-1"></a>

Before you start, install Docker and NVIDIA drivers:
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 460+

The Python environment should be based on Python 3.7 and this notebook can be loaded in Jupyer-Lab on OSx (M1 chip not supported), Windows, or Linux.

In [ ]:
# You may have to change the kernel of the notebook to your virtual environment
import sys
if sys.version_info[1]!=7:
    raise Exception("Must be Python 3.7, got {}".format(sys.version))

In [ ]:
# Set up a site-package in the local directory for python packages
import os
!mkdir site-packages

currentDirList = !pwd
currentDir = currentDirList[0]
currentPathsList = !echo $PATH
currentPaths = currentPathsList[0]
sitePackagesDir = os.path.join(currentDir, 'site-packages')
newPaths = ':'.join([currentPaths[0], sitePackagesDir, os.path.join(sitePackagesDir, 'bin')])
%env PATH={newPaths}

In [ ]:
# Install the Rendered.ai SDK `anatools` and other dependencies for this notebook
if not os.path.isdir(os.path.join(sitePackagesDir, 'anatools')):
    !{sys.executable} -m pip install anatools -t site-packages
    !{sys.executable} -m pip install tensorflow -t site-packages
    !{sys.executable} -m pip install matplotlib==3.3.3 -t site-packages

In [ ]:
## 2 Install the TAO Launcher <a class="anchor" id="head-2"></a>
TAO interfaces with NGC by running a Docker locally that the tao command uses to prepare data, download pretrained
models, train models, etc. The TAO launcher is a python package distributed as a python wheel listed in the `nvidia-pyindex` python index.

In [ ]:
# Skip this step if you have already installed the tao launcher.
if not os.path.isdir(os.path.join(sitePackagesDir, 'bin')):
    !{sys.executable} -m pip install nvidia-pyindex -t site-packages
    !{sys.executable} -m pip install nvidia-tao -t site-packages

# View the version of the TAO launcher
!tao info

### 2.2 Configure the local environment to use TAO <a class="anchor" id="head-2-2"></a>

The cell below maps the project directory on your local host to a workspace directory in the TAO docker instance, so that the data and the results are mapped from in and out of the docker. For more information please refer to the [launcher instance](https://docs.nvidia.com/tao/tao-toolkit/text/tao_launcher.html) in the user guide.

When running this cell on AWS, update the drive_map entry with the dictionary defined below, so that you don't have permission issues when writing data into folders created by the TAO docker.

```json
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tao-experiments"
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["TAO_SPECS_DIR"]
        },
    ],
    "DockerOptions": {
        "user": "1000:1000"
    },
    # set gpu index for tao-converter
    "Envs": [
        {"variable": "CUDA_VISIBLE_DEVICES", "value": os.getenv("GPU_INDEX")},
    ]
}
```

The following global environment variables identify the location of the project and the data. Note the `$LOCAL_PROJECT_DIR` is the path to the local workspace.

In [ ]:
import os

print("Please replace the variables with your own.")
%env GPU_INDEX=0
%env KEY=renderedai

os.environ["USR"] = str(os.getuid())  # use '1000' for windows
os.environ["GRP"] = str(os.getgid())  # use '1000' for windows

projectDir = os.path.abspath(os.path.join(os.getcwd(), '..') )  # 'local/path/to/tao_experiments'
os.environ["LOCAL_PROJECT_DIR"] = projectDir
!echo $LOCAL_PROJECT_DIR

experimentDir = os.getcwd()
os.environ["LOCAL_EXPERIMENT_DIR"] = experimentDir

experimentName =  os.path.basename(experimentDir)

renderedDataDir =  'data_rendered'
os.environ['LOCAL_DATA_DIR'] = os.path.join(projectDir, renderedDataDir)

In [ ]:
# Mapping up the local directories to the TAO docker.

taoDockerProjectDir = "/workspace/tao-experiments/"
taoDockerExperimentDir = taoDockerProjectDir + experimentName
taoDockerDataDir = taoDockerProjectDir + renderedDataDir
os.environ["TAO_DATA_DIR"]=taoDockerDataDir

# The sample spec files are present in the same path as the downloaded samples.
# Set this path if you don't run the notebook from the samples directory.  
localSepcsDir = os.path.join(experimentDir, "specs")
os.environ["LOCAL_SPECS_DIR"] = localSepcsDir
os.environ["TAO_SPECS_DIR"] = os.path.join(taoDockerExperimentDir, 'specs')

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR

# Set the MPLCONFIGDIR environment variable to a writable directory to speed up the import of Matplotlib and to better support multiprocessing.
os.environ["TAO_MPLCONFIG_DIR"] = os.path.join(taoDockerExperimentDir, "matplotlib_config")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": taoDockerProjectDir
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["TAO_SPECS_DIR"]
        },
        {
            "source": os.environ["LOCAL_DATA_DIR"],
            "destination": os.environ["TAO_DATA_DIR"]
        }
    ],
    "DockerOptions": {
        "user": "{}:{}".format(os.environ["USR"], os.environ["GRP"])
    },
    # set gpu index for tao-converter
    "Envs": [
        {"variable": "CUDA_VISIBLE_DEVICES", "value": os.getenv("GPU_INDEX")},
        {"variable": "MPLCONFIGDIR", "value": os.getenv("TAO_MPLCONFIG_DIR")}
    ]
}

In [ ]:
# Writing the mounts file.
import json
mounts_file = os.path.expanduser("~/.tao_mounts.json")
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [ ]:
# Check the state of your mounts file
!cat ~/.tao_mounts.json

## 3 Download Training Dataset from Rendered.ai <a class="anchor" id="head-3"></a>

### 3.1 Authenticate with Rendered using the anatools SDK <a class="anchor" id="head-3-1"></a>

In [ ]:
import anatools
ana = anatools.client()
# Log in to Rendered.ai; Register at https://www.rendered.ai/waitlist.html 

In [ ]:
# Choose a workspace
try:
    workspacesByName={ws['name']:ws for ws in ana.get_workspaces()}
except AttributeError:
    print("Login Expired")
    ana = anatools.client()
    workspacesByName={ws['name']:ws for ws in ana.get_workspaces()}

workspace = workspacesByName['NVIDIA TAO']  ### CHANGE THIS FOR YOUR WORKSPACE IN YOUR ACCOUNT
print(workspace)

In [ ]:
# Choose a dataset for training tao and add the name to the environment
datasets = ana.get_datasets(workspaceId=workspace['workspaceId'])

datasetsByName = {ds['name']:ds for ds in datasets}
print('DATASET NAME - DESCRIPTION')
for n,ds in datasetsByName.items():
    print('{} - {}'.format(n, ds['description']))

datasetName = '1000runs_15objectScene'
os.environ['DATASET_NAME'] = datasetName

Rendered.ai supports various annotation formats including COCO and KITTI. These are application specific annotation formats that can be generated with `anatools`. To integrate with NVIDIA TAO faster_rcnn, here we use the KITTI annotaion format.

In [ ]:
# KITTI label generation utility
def createKittiLabels(mapfilepath, dataset_name = os.environ['DATASET_NAME']):
    datasetdir = os.path.join(os.environ['LOCAL_DATA_DIR'], dataset_name, 'output')
    kittiDir = os.path.join(os.environ['LOCAL_DATA_DIR'], dataset_name, 'output', 'kitti_labels')
    if not os.path.isdir(kittiDir):
        os.mkdir(kittiDir)
    anatools.Annotations().dump_kitti(datasetdir, kittiDir, mapfilepath)

In [ ]:
# Download the dataset and the default object mapping files
# Other availailable mappings can be used: https://github.com/Rendered-ai/ana/tree/main/ana/channels/example/mappings
mappingsDir = os.path.join(os.environ['LOCAL_DATA_DIR'], 'mappings')
mappingsFileName = os.path.join(mappingsDir, 'default.yml')
if os.path.isfile(mappingsFileName):
    print("Found mapping file " + mappingsFileName)
else:
    wget_command = '-P {} https://raw.githubusercontent.com/Rendered-ai/ana/main/ana/channels/example/mappings/default.yml'.format(mappingsDir)
    !wget {wget_command}
    print("Downloaded " + mappingsFileName)

dataDir = os.path.join(os.environ['LOCAL_DATA_DIR'], os.environ['DATASET_NAME'])
if os.path.isdir(dataDir):
    print("Found dataset " + datasetName)
else:
    print("This might take some time...")
    print("Downloading " + dataDir)
    datasetZipFile = ana.download_dataset(datasetsByName[datasetName]['datasetId'], workspace['workspaceId'])
    
    print("Unzipping...")
    !mkdir -p $dataDir
    !unzip -q -u $datasetZipFile -d $dataDir
    
    # Check the dataset is present
    unzipSuccess = os.path.isdir(os.path.join(os.environ['LOCAL_DATA_DIR'], os.environ['DATASET_NAME'], 'output', 'images'))
    if unzipSuccess:
        os.remove(datasetZipFile)
    
    createKittiLabels(matppingsFileName, datasetName)

### 3.2 Verify Dataset <a class="anchor" id="head-3-2"></a>

In [ ]:
#Number of Images
num_images = len(os.listdir(os.path.join(dataDir, 'output', 'images')))
print("Number of images in the dataset. {}".format(num_images))

num_labels = len(os.listdir(os.path.join(dataDir, 'output', 'kitti_labels')))
print("Number of labels in the dataset. {}\n".format(num_labels))

In [ ]:
# Sample kitti labels.
filename = os.listdir(os.path.join(os.environ['LOCAL_DATA_DIR'], os.environ['DATASET_NAME'], 'output', 'kitti_labels'))[0]
print(filename)
with open(os.path.join(os.environ['LOCAL_DATA_DIR'], os.environ['DATASET_NAME'], 'output', 'kitti_labels', filename)) as fin:
    for i in range(5):
        print(fin.readline())

 ## 4. Prepare dataset and pretrained model <a class="anchor" id="head-4"></a>

### 4.1 Prepare tfrecords from kitti format dataset <a class="anchor" id="head-4-1"></a>

* Update the template tfrecords spec file to take in the synthetic kitti format dataset
* Create the tfrecords using the dataset_convert
* TFRecords only need to be generated once.

In [ ]:
fin = open(os.path.join(localSepcsDir, 'frcnn_tfrecords_trainval.txt'))
with open(os.path.join(localSepcsDir, 'frcnn_tfrecords_trainval_tmp.txt'), 'w') as fout:
    for line in fin:
        fout.write(line.replace('datasetName', os.environ['DATASET_NAME']))
fin.close()

print("TFrecords conversion spec file for training")
!cat $LOCAL_SPECS_DIR/frcnn_tfrecords_trainval_tmp.txt

In [ ]:
!tao faster_rcnn dataset_convert \
  --gpu_index $GPU_INDEX \
  -d $TAO_SPECS_DIR/frcnn_tfrecords_trainval_tmp.txt \
  -o $TAO_DATA_DIR/$DATASET_NAME/output/tfrecords/trainval

# View the TF records directory
#!ls -rlt $LOCAL_DATA_DIR/$DATASET_NAME/output/tfrecords | tail

In [ ]:
# Take a look at a tfrecord
import tensorflow as tf

tfrecordDir = os.path.join(os.environ['LOCAL_DATA_DIR'], os.environ['DATASET_NAME'], 'output', 'tfrecords')
tfrecord = None
for tfrecordfile in os.listdir(tfrecordDir):
    print(tfrecordfile)
    for example in tf.compat.v1.python_io.tf_record_iterator(os.path.join(tfrecordDir, tfrecordfile)):
        tfrecord = tf.train.Example.FromString(example)
        print(tfrecord)
    if tfrecord:
        break

 ### 4.2 Download pre-trained model <a class="anchor" id="head-4-2"></a>

To get the ResNet-18 Backbone from NGC. Please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.

In [ ]:
# Install NGC CLI on the local machine.

## Download and install
if os.path.isdir(os.path.join(projectDir, 'ngccli')):
    print("Found existing model " + datasetName)
    !rm -rf $LOCAL_PROJECT_DIR/ngccli/*
else:
    print("Downloading...")
    %env CLI=ngccli_cat_linux.zip
    !mkdir -p $LOCAL_PROJECT_DIR/ngccli 
    !wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
    !unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
    !rm $LOCAL_PROJECT_DIR/ngccli/*.zip

In [ ]:
os.environ["PATH"]="{}/ngccli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))
!ngc registry model list nvidia/*pretrained_object_detection*

In [ ]:
# Download model from NGC.
!ngc registry model download-version nvidia/tao/pretrained_object_detection:resnet18

In [ ]:
# Verify ResNet-18 download
!ls -rlt $LOCAL_EXPERIMENT_DIR/pretrained_object_detection_vresnet18

 ## 5. Run TAO Training <a class="anchor" id="head-5"></a>

 ## 5.1. Provide training specification <a class="anchor" id="head-5-1"></a>
 The NVIDIA TAO commands used throughout this notebook use the spec files in $LOCAL_SPECS_DIR. The following helper funciton clones the file used for training, evaluation and making inferences. The parameters within are described in the [TAO Toolkit Documentation](https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/detectnet_v2.html#creating-a-configuration-file).

In [ ]:
# Reset the tao model file to use the resnet18 model to detect all the objects as individual classes.
def experimentSpec():
    fin = open(os.path.join(localSepcsDir, 'alltoys_spec_resnet18.txt'))
    with open(os.path.join(localSepcsDir, 'alltoys_spec_resnet18_tmp.txt'), 'w') as fout:
        for line in fin:
            fout.write(line.replace('$KEY', os.environ['KEY']).replace('datasetName', os.environ['DATASET_NAME']))
    fin.close()

experimentSpec()

!cat $LOCAL_SPECS_DIR/alltoys_spec_resnet18_tmp.txt

In [ ]:
# You can restart with the last completed epoch: uncomment line 104 in the spec file
!tao faster_rcnn train --gpu_index $GPU_INDEX -e $TAO_SPECS_DIR/alltoys_spec_resnet18_tmp.txt

In [ ]:
print("For resume training from checkpoint, please uncomment and run this instead. Change/Add the 'resume_from_model' field in the spec file.")
#!tao faster_rcnn train --gpu_index $GPU_INDEX -e $TAO_SPECS_DIR/alltoys_spec_resnet18_tmp.txt

In [ ]:
print('Model for each epoch:')
print('---------------------')
!ls -lht *$DATASET_NAME*

 ## 6. Evaluate Model on Various Datasets <a class="anchor" id="head-6"></a>

In [ ]:
# Evaluation sets 
    
# Look at the datasets in your workspace
for ds in datasets:
    print('{}: {}'.format(ds['name'],ds['description']))

# Choose test sets
datasetNameObstruction10 = '200runs_10objectScene'
datasetNameObstruction40 = '200runs_40objectScene'
datasetNameObstruction70 = '200runs_70objectScene'
datasetNameObstruction100 = '200runs_100objectScene'
datasetNameObstruction200 = '200runs_200objectScene'

testDatasets = [datasetNameObstruction10, datasetNameObstruction40, datasetNameObstruction70, datasetNameObstruction100]

print('\nUsing the following datasets...')
for testDataset in testDatasets:
    print('{}: {}\n'.format(testDataset, datasetsByName[testDataset]))

In [ ]:
# Download Test Datasets
for testDataset in testDatasets:
    dataDir = os.path.join(projectDir, renderedDataDir, testDataset)
    if os.path.isdir(dataDir):
        print("Found dataset " + testDataset)
    else:
        print("Downloading " + dataDir)
        try:
            datasetZipFile = ana.download_dataset(datasetsByName[testDataset]['datasetId'], workspace['workspaceId'])
        except AttributeError:
            print("Login Expired")
            ana = anatools.client()
            print("Downloading " + dataDir)
            datasetZipFile = ana.download_dataset(datasetsByName[testDataset]['datasetId'], workspace['workspaceId'])

        print("Unzipping ...")
        !mkdir -p $dataDir
        !unzip -q -u $datasetZipFile -d $dataDir

        # Check the dataset is present
        unzipSuccess = os.path.isdir(os.path.join(os.environ['LOCAL_DATA_DIR'], testDataset, 'output', 'images'))
        if unzipSuccess:
            os.remove(datasetZipFile)

        createKittiLabels(mappingsFileName, testDataset)

In [ ]:
#Verify test sets
for testDataset in testDatasets:
    print(testDataset)
    dataDir = os.path.join(os.environ['LOCAL_DATA_DIR'], testDataset)
    num_images = len(os.listdir(os.path.join(dataDir, 'output', 'images')))
    print("Number of images in the dataset. {}".format(num_images))

    #Kitti Labels
    num_labels = len(os.listdir(os.path.join(dataDir, 'output', 'kitti_labels')))
    print("Number of labels in the dataset. {}\n".format(num_labels))

In [ ]:
#Generate tensorflow records
for testDataset in testDatasets:
    print("Createing TFrecords for test set: " + testDataset)
    fin = open(os.path.join(localSepcsDir, 'frcnn_tfrecords_eval.txt'))
    with open(os.path.join(localSepcsDir, 'frcnn_tfrecords_eval_tmp.txt'), 'w') as fout:
        for line in fin:
            fout.write(line.replace('datasetName', testDataset))
    fin.close()

    relativeDatasetDir =  os.path.join('data_rendered', testDataset, "output")
    os.environ["TAO_TESTSET_DIR"] = os.path.join(taoDockerProjectDir, relativeDatasetDir)

    !tao faster_rcnn dataset_convert --gpu_index $GPU_INDEX \
       -d $TAO_SPECS_DIR/frcnn_tfrecords_eval_tmp.txt -o $TAO_TESTSET_DIR/tfrecords/eval

In [ ]:
#Update the experiment spec file to use the evaluation images and tfrecords in dataset_config[data_sources]
def evalModel(testDataset):
    # Reset the experiment spec so the dataset_config parameters are for the training name ('datsetName')
    experimentSpec()
    with open(os.path.join(os.environ["LOCAL_SPECS_DIR"], 'alltoys_spec_resnet18_tmp.txt')) as fin:
        filedata = fin.read()

    existingdata = '{}/output'.format(datasetName)
    newdata = '{}/output'.format(testDataset)
    filedata = filedata.replace(existingdata, newdata)

    with open(os.path.join(os.environ["LOCAL_SPECS_DIR"], 'alltoys_spec_resnet18_tmp.txt'), 'w') as fout:
        fout.write(filedata)

    !tao faster_rcnn evaluate --gpu_index $GPU_INDEX -e $TAO_SPECS_DIR/alltoys_spec_resnet18_tmp.txt

In [ ]:
def estimate_obstruction(kitti_dir):
    truncation = 0
    occlusion = 0
    nobjects = 0
    for kittifilename in os.listdir(kitti_dir):
        with open(os.path.join(kitti_dir, kittifilename), 'r') as af:
            labels = af.readlines()

        for label in labels:
            features = label.split(' ')
            truncation += int(features[1])
            occlusion += int(features[2])
        nobjects += len(labels)
    obstruction = (truncation + occlusion)/nobjects
    return obstruction

In [ ]:
testDataset = testDatasets[0]
evalModel(testDataset)
kittiDir = os.path.join(projectDir, renderedDataDir, testDataset, 'output', 'kitti_labels')
print("\nAverage Object Obstruction: {}".format(estimate_obstruction(kittiDir)))

In [ ]:
testDataset = testDatasets[1]
evalModel(testDataset)
kittiDir = os.path.join(projectDir, renderedDataDir, testDataset, 'output', 'kitti_labels')
print("\nAverage Object Obstruction: {}".format(estimate_obstruction(kittiDir)))

In [ ]:
testDataset = testDatasets[2]
evalModel(testDataset)
kittiDir = os.path.join(projectDir, renderedDataDir, testDataset, 'output', 'kitti_labels')
print("\nAverage Object Obstruction: {}".format(estimate_obstruction(kittiDir)))

In [ ]:
testDataset = testDatasets[3]
evalModel(testDataset)
kittiDir = os.path.join(projectDir, renderedDataDir, testDataset, 'output', 'kitti_labels')
print("\nAverage Object Obstruction: {}".format(estimate_obstruction(kittiDir)))

### Results
The following plot of the evaluation results shows the relationship between detection accuracy how much the objects are obstructed. A linear regression analysis indicates mAP0.5 drops 0.78% for each percent obstruction increase.
<img align="center" src="https://renderedai-public-assets.s3.us-west-2.amazonaws.com/Accuracy_vs_Obstruction.png" width="540"> 


 ## 7. Visualize Inferences <a class="anchor" id="head-7"></a>

The `inference` tool of tao faster_rnn generates inferences on a dataset.

In [ ]:
# Running inference, see the `inference_config` parameters in the spec file used here.
!tao faster_rcnn inference --gpu_index $GPU_INDEX -e $TAO_SPECS_DIR/alltoys_spec_resnet18_tmp.txt
print('The inferences are for "'+testDataset+'" - see "inference_config" of spec file: "alltoys_spec_resnet18_tmp.txt"')

The `inference` tool produces two outputs.
1. Overlain images in `$LOCAL_EXPERIMENT_DIR/inference_results_imgs`
2. Frame by frame bbox labels in kitti format located in `$LOCAL_EXPERIMENT_DIR/inference_dump_labels`

*Note: To run inferences for a single image, simply replace the path to the -i flag in `inference` command with the path to the image.*


In [ ]:
# Simple grid visualizer
%matplotlib inline
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path)
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img)

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'inference_results_imgs' # relative path from $LOCAL_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 6 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

In [ ]:
# Cleanup
!rm $LOCAL_SPECS_DIR/*_tmp.txt

### Congradulations!
Inspecting the bboxes can provide evidence for various hypotheses for further experiments.  For instance the skateboard is the most affeted by obstruction. Perhaps there is one particular style of skateboard that is failing detection or causing false positive. Armed with a hypothesis, the scientist can update the graph on Rendered.ai and add to the training set.